In [1]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup


In [2]:
def load_and_clean_data(file_path):
    df = pd.read_excel(file_path).iloc[3:]
    df.columns = df.iloc[0]
    df = df.iloc[1:].reset_index(drop=True)
    return df

def filter_data(df):
    excluded_keys = ['------', '892000', '892500', '734600']
    df = df[~df['Schlüssel'].isin(excluded_keys)]
    df['Schlüssel'] = df['Schlüssel'].astype(int)
    return df

def categorize_crimes(df):
    crime_conditions = [
        (df['Schlüssel'] >= 10000) & (df['Schlüssel'] < 20000),
        (df['Schlüssel'] >= 20000) & (df['Schlüssel'] <= 30000),
        (df['Schlüssel'] >= 110000) & (df['Schlüssel'] <= 145000),
        (df['Schlüssel'] >= 210000) & (df['Schlüssel'] <= 219050),
        ((df['Schlüssel'] >= 220000) & (df['Schlüssel'] <= 225000)) | (df['Schlüssel'] == 655100),
        (df['Schlüssel'] >= 230000) & (df['Schlüssel'] <= 239540),
        (df['Schlüssel'] >= 621100) & (df['Schlüssel'] <= 621120),
        (df['Schlüssel'] >= 641000) & (df['Schlüssel'] <= 641040),
        (df['Schlüssel'] == 670021),
        (df['Schlüssel'] >= 725400) & (df['Schlüssel'] <= 725410)
    ]
    
    crime_labels = [
        'Murder', 'Homicide', 'Sex-Related Offenses', 'Robbery', 'Assault',
        'Deprivation of Personal Liberty', 'Resisting Arrest', 'Arson', 
        'Abandonment', 'Smuggling of Persons'
    ]
    
    df['Crime'] = np.select(crime_conditions, crime_labels, default='Other')
    return df

def preprocess_data(df, total_deutschland, total_nichtdeutsche):
    df = df[df['Fall-status'] == 'insg.']
    df = df.drop(columns=['Schlüssel', 'Straftat', 'Fall-status'])
    
    for column in df.columns:
        if column != 'Crime':
            df[column] = pd.to_numeric(df[column], errors='coerce').fillna(0)
    
    df_gr = df.groupby('Crime').sum()
    df_gr = df_gr.loc[:, (df_gr != 0).any(axis=0)]
    df_gr = df_gr.loc[(df_gr != 0).any(axis=1)]
    
    df_gr['Deutsche_Opfer_norm'] = df_gr['Deutschland'] / total_deutschland
    df_gr['Nichtdeutsche_Opfer_norm'] = df_gr['Nichtdeutsche insgesamt'] / total_nichtdeutsche
    
    return df_gr

def process_data(file_path, year, total_deutschland, total_nichtdeutsche):
    df = load_and_clean_data(file_path)
    df = filter_data(df)
    df = categorize_crimes(df)
    df_gr = preprocess_data(df, total_deutschland, total_nichtdeutsche)
    df_gr = df_gr.rename(columns=lambda x: f"{x}_{year}")
    return df_gr

data_files = {
    2023: ('BU-O-04-T911-O-Staatsangehoerigkeiten_xls_2023.xlsx', 71855657, 12751359),
    2022: ('BU-O-04-T911-O-Staatsangehoerigkeiten_xls_2022.xlsx', 72034650, 12324195),
    2021: ('BU-O-04-T911-O-Staatsangehoerigkeiten_xls_2021.xlsx', 72344071, 10893053),
    2020: ('BU-O-04-T911-O-Staatsangehoerigkeiten_xls_2020.xlsx', 72569978, 10585053),
    2019: ('BU-O-04-T911-O-Staatsangehoerigkeiten_xls_2019.xlsx', 72768689, 10398022)
}

data_gr_dict = {year: process_data(file_path, year, total_deutschland, total_nichtdeutsche) 
                for year, (file_path, total_deutschland, total_nichtdeutsche) in data_files.items()}

data_combined = pd.concat(data_gr_dict.values(), axis=1)


/var/folders/qg/sgj3q0zj6xvcgvl5h1k6w_nm0000gn/T/ipykernel_79233/3113588624.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Schlüssel'] = df['Schlüssel'].astype(int)
/var/folders/qg/sgj3q0zj6xvcgvl5h1k6w_nm0000gn/T/ipykernel_79233/3113588624.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Schlüssel'] = df['Schlüssel'].astype(int)
/var/folders/qg/sgj3q0zj6xvcgvl5h1k6w_nm0000gn/T/ipykernel_79233/3113588624.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [3]:

# Create a DataFrame for population data
data_population = pd.DataFrame({
    'Jahr': ['2019', '2020', '2021', '2022', '2023'],
    'Deutsche_total': [None] * 5,
    'Nichtdeutsche_total': [None] * 5
})

# Populate the DataFrame using a loop
for year, (file_path, deutsche_total, nichtdeutsche_total) in data_files.items():
    data_population.loc[data_population['Jahr'] == str(year), 'Deutsche_total'] = deutsche_total
    data_population.loc[data_population['Jahr'] == str(year), 'Nichtdeutsche_total'] = nichtdeutsche_total

# Fill NaN values with 0 in the combined data
data_combined = data_combined.fillna(0)

# Select columns for the CSV output
columns_to_export = [
    'Deutsche_Opfer_norm_2019', 'Nichtdeutsche_Opfer_norm_2019',
    'Deutsche_Opfer_norm_2020', 'Nichtdeutsche_Opfer_norm_2020',
    'Deutsche_Opfer_norm_2021', 'Nichtdeutsche_Opfer_norm_2021',
    'Deutsche_Opfer_norm_2022', 'Nichtdeutsche_Opfer_norm_2022',
    'Deutsche_Opfer_norm_2023', 'Nichtdeutsche_Opfer_norm_2023'
]

# Save the selected columns to a CSV file
data_combined[columns_to_export].to_csv('data_D_NichtD_proportion.csv')

# Save the population data to a CSV file
data_population.to_csv('data_D_NichtD_population.csv')


In [4]:
# Define the columns to save
columns_to_save = [
    'Deutsche_Opfer_norm_2019', 'Nichtdeutsche_Opfer_norm_2019',
    'Deutsche_Opfer_norm_2020', 'Nichtdeutsche_Opfer_norm_2020',
    'Deutsche_Opfer_norm_2021', 'Nichtdeutsche_Opfer_norm_2021',
    'Deutsche_Opfer_norm_2022', 'Nichtdeutsche_Opfer_norm_2022',
    'Deutsche_Opfer_norm_2023', 'Nichtdeutsche_Opfer_norm_2023'
]

# Create a new DataFrame with only the specified columns
df_to_pivot = data_combined[columns_to_save]

# Use pd.melt to pivot the DataFrame
df_melted = pd.melt(df_to_pivot.reset_index(), id_vars='Crime', value_vars=columns_to_save,
                    var_name='Year_Variable', value_name='Value')

# Extract the year and the category from the 'Year_Variable' column
df_melted['Year'] = df_melted['Year_Variable'].str.extract(r'(\d{4})$')
df_melted['Category'] = df_melted['Year_Variable'].str.replace(r'_\d{4}$', '')

# Rename categories for clarity
df_melted['Category'] = df_melted['Category'].replace({
    'Deutsche_Opfer_norm': 'Deutsche Opfer der Gewalt',
    'Nichtdeutsche_Opfer_norm': 'Nichtdeutsche Opfer der Gewalt'
})

# Remove the 'Year_Variable' column
df_melted = df_melted.drop(columns=['Year_Variable'])

# Rearrange the columns for better readability
df_melted = df_melted[['Crime', 'Year', 'Category', 'Value']]

# Save the tidy DataFrame to a CSV file
df_melted.to_csv('data_tidy.csv', index=False)

# Create a pivot table with 'Year' and 'Crime' as the index and 'Category' as the columns
df_pivoted_back = df_melted.pivot_table(index=['Year', 'Crime'], columns='Category', values='Value')

# Reset the index to make 'Year' and 'Crime' columns again
df_pivoted_back = df_pivoted_back.reset_index()

# Save the pivoted DataFrame to a CSV file
df_pivoted_back.to_csv('data_pivoted_back.csv', index=False)


/var/folders/qg/sgj3q0zj6xvcgvl5h1k6w_nm0000gn/T/ipykernel_79233/3848557408.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df_melted['Category'] = df_melted['Year_Variable'].str.replace(r'_\d{4}$', '')


In [5]:
# Function to normalize data within each group for combined columns
def combined_min_max_normalize(group, columns):
    combined_min = group[columns].min().min()
    combined_max = group[columns].max().max()
    result = group.copy()
    for column in columns:
        result[column] = (group[column] - combined_min) / (combined_max - combined_min)
    return result

# Fill NaN values with 0 in the pivoted DataFrame
df_pivoted_back = df_pivoted_back.fillna(0)

# Define columns to normalize together
columns_to_normalize = ['Deutsche Opfer der Gewalt', 'Nichtdeutsche Opfer der Gewalt']

# Apply combined Min-Max normalization within each crime category
df_normalized = df_pivoted_back.groupby('Crime').apply(
    combined_min_max_normalize, columns=columns_to_normalize
).reset_index(drop=True)

# Save the normalized DataFrame to a CSV file
df_normalized.to_csv('data_normalized.csv', index=False)


/var/folders/qg/sgj3q0zj6xvcgvl5h1k6w_nm0000gn/T/ipykernel_79233/81259306.py:17: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_normalized = df_pivoted_back.groupby('Crime').apply(


In [6]:

# Define the columns to delete
columns_to_delete = [
    'Opfer insgesamt_2023',  'Nichtdeutsche insgesamt_2023',
    'Deutsche_Opfer_norm_2023', 'Nichtdeutsche_Opfer_norm_2023',
    'Opfer insgesamt_2022', 'Nichtdeutsche insgesamt_2022',
    'Deutsche_Opfer_norm_2022', 'Nichtdeutsche_Opfer_norm_2022',
    'Opfer insgesamt_2021',  'Nichtdeutsche insgesamt_2021',
    'Deutsche_Opfer_norm_2021', 'Nichtdeutsche_Opfer_norm_2021',
    'Opfer insgesamt_2020',  'Nichtdeutsche insgesamt_2020',
    'Deutsche_Opfer_norm_2020', 'Nichtdeutsche_Opfer_norm_2020',
    'Opfer insgesamt_2019',  'Nichtdeutsche insgesamt_2019',
    'Deutsche_Opfer_norm_2019', 'Nichtdeutsche_Opfer_norm_2019'
]

# Drop the unwanted columns
data_combined_countries = data_combined.drop(columns=columns_to_delete)

data_combined_countries=data_combined_countries.reset_index(names='Crime')

In [7]:
# Define lists of countries
european_countries = ['Bulgarien', 'Frankreich', 'Griechenland', 'Italien', 'Rumänien', 
                      'Niederlande', 'Polen', 'Portugal', 'Rumänien', 'Spanien', 
                      'Ungarn', 'Österreich', 'Serbien']

refugee_countries = ['Afghanistan', 'Irak', 'Kosovo', 'Syrien', 'Ukraine', 'Türkei']
other_countries = ['China', 'Indien', 'Russische Föderation']

# Define function to generate columns for each year
def generate_yearly_columns(country_list, year):
    return [f"{country}_{year}" for country in country_list]

# Generate yearly columns
years = [2023, 2022, 2021, 2020, 2019]
european_columns = {year: generate_yearly_columns(european_countries, year) for year in years}
refugee_columns = {year: generate_yearly_columns(refugee_countries, year) for year in years}
other_columns = {year: generate_yearly_columns(other_countries, year) for year in years}

# Define function to sum columns for each category and year
def sum_columns_for_year(dataframe, columns_dict, category_name):
    for year, columns in columns_dict.items():
        dataframe[f'{category_name}_{year}'] = dataframe[columns].sum(axis=1)
    return 
# Apply the function to sum columns for each category
sum_columns_for_year(data_combined_countries, european_columns, 'Europäische Länder')
sum_columns_for_year(data_combined_countries, refugee_columns, 'Asylsuchende')
sum_columns_for_year(data_combined_countries, other_columns, 'Andere Länder')


In [8]:
data_combined_countries

3,Crime,Deutschland_2023,Afghanistan_2023,Ägypten_2023,Albanien_2023,Algerien_2023,Andorra_2023,Angola_2023,Antigua und Barbuda_2023,Äquatorialguinea_2023,...,Asylsuchende_2023,Asylsuchende_2022,Asylsuchende_2021,Asylsuchende_2020,Asylsuchende_2019,Andere Länder_2023,Andere Länder_2022,Andere Länder_2021,Andere Länder_2020,Andere Länder_2019
0,Abandonment,90.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,5.0,4,10,1.0,6,0.0,0,0,0.0,0
1,Arson,56.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,2,0,2.0,0,0.0,0,0,0.0,0
2,Assault,1126300.0,33264.0,2641.0,3840.0,6251.0,4.0,433.0,4.0,27.0,...,200398.0,164760,129522,142202.0,142233,13130.0,11032,9784,10817.0,10330
3,Deprivation of Personal Liberty,886846.0,9228.0,1052.0,1393.0,878.0,0.0,89.0,0.0,16.0,...,88780.0,72795,57950,53187.0,50527,6091.0,5170,3985,3918.0,3793
4,Homicide,2869.0,80.0,0.0,21.0,20.0,0.0,0.0,0.0,0.0,...,561.0,555,500,599.0,484,45.0,26,33,35.0,26
5,Murder,1382.0,18.0,0.0,10.0,2.0,0.0,0.0,0.0,2.0,...,246.0,194,150,200.0,264,12.0,24,16,18.0,16
6,Resisting Arrest,200616.0,4.0,6.0,0.0,4.0,0.0,0.0,0.0,0.0,...,314.0,272,230,244.0,184,46.0,34,28,22.0,14
7,Robbery,99544.0,2021.0,230.0,242.0,490.0,0.0,16.0,0.0,8.0,...,11556.0,8402,6596,7175.0,7178,1194.0,860,630,773.0,747
8,Sex-Related Offenses,181303.0,1025.0,110.0,207.0,87.0,0.0,15.0,0.0,0.0,...,8739.0,7568,5256,5402.0,5063,1016.0,1036,774,825.0,756
9,Smuggling of Persons,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2,0,0.0,0,0.0,0,0,0.0,0


In [9]:

# Function to fetch and parse the HTML content
def fetch_webpage(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None
    return BeautifulSoup(response.content, 'html.parser')

# Function to find and parse the table into a DataFrame
def extract_table_from_soup(soup):
    table = soup.find('table')  # Modify this if needed to locate the specific table
    if table:
        df = pd.read_html(str(table))[0]
        return df
    else:
        print("No table found.")
        return None

# Function to clean and filter the DataFrame
def clean_and_filter_dataframe(df):
    df = df.rename(columns={'Staats­angehörig­keit': 'Staatsangehörigkeit'})
    df = df[['Staatsangehörigkeit', '2023', '2022', '2021', '2020', '2019']]
    
    # Correct specific country names
    corrections = {
        'Ukraine3': 'Ukraine',
        'Russische\xa0Föderation': 'Russische Föderation'
    }
    df['Staatsangehörigkeit'] = df['Staatsangehörigkeit'].replace(corrections)
    
    # Define country groups
    all_countries = ['Rumänien', 'Polen', 'Italien', 'Bulgarien', 'Kroatien', 'Griechenland', 
                     'Ungarn', 'Spanien', 'Österreich', 'Niederlande', 'Frankreich', 'Portugal',
                     'Türkei', 'Ukraine', 'Russische Föderation', 'Kosovo', 'Serbien',
                     'Vereinigte Staaten (USA)', 'Syrien', 'Afghanistan', 'Irak', 'Indien', 'China']
    
    # Filter rows based on country
    df = df[df['Staatsangehörigkeit'].isin(all_countries)]
    
    # Clean and convert data types
    df[['2023', '2022', '2021', '2020', '2019']] = df[['2023', '2022', '2021', '2020', '2019']].replace(r'\xa0', '', regex=True).astype(int)
    
    return df

# Main process function
def process_web_data(url):
    soup = fetch_webpage(url)
    if soup:
        df = extract_table_from_soup(soup)
        if df is not None:
            df = clean_and_filter_dataframe(df)
            df.to_csv('states_germany.csv', index=False)
            print("Data saved successfully.")
        else:
            print("Failed to process the DataFrame.")
    else:
        print("Failed to fetch or parse webpage content.")
    return df
# URL of the target webpage
url = 'https://www.destatis.de/DE/Themen/Gesellschaft-Umwelt/Bevoelkerung/Migration-Integration/Tabellen/auslaendische-bevoelkerung-staatsangehoerigkeit-jahre.html'
df=process_web_data(url)

Data saved successfully.


In [10]:


# Define your countries lists
european_countries = ['Bulgarien', 'Frankreich', 'Griechenland', 'Italien', 'Rumänien', 
                      'Niederlande', 'Polen', 'Portugal', 'Rumänien', 'Spanien', 
                      'Ungarn', 'Österreich', 'Serbien']
refugee_countries = ['Afghanistan', 'Irak', 'Kosovo', 'Syrien', 'Ukraine', 'Türkei']
other_countries = ['China', 'Indien', 'Russische Föderation']

# Function to sum values for a given group of countries
def sum_country_data(df, countries, label):
    df_filtered = df[df['Staatsangehörigkeit'].isin(countries)]
    summed_values = df_filtered.drop(columns=['Staatsangehörigkeit']).sum().to_frame().T
    summed_values['Staatsangehörigkeit'] = label
    return summed_values

# Sum values for each group and append to the DataFrame
def append_summed_values(df, european_countries, refugee_countries, other_countries):
    summed_european = sum_country_data(df, european_countries, 'Europäische Länder')
    summed_refugee = sum_country_data(df, refugee_countries, 'Asylsuchende')
    summed_other = sum_country_data(df, other_countries, 'Andere Länder')

    # Concatenate summed rows with the original DataFrame
    return pd.concat([df, summed_european, summed_refugee, summed_other], ignore_index=True)

# Function to add a new row
def add_new_row(df, new_row):
    df.loc[len(df)] = new_row
    return df

# Assuming df is already defined
df = append_summed_values(df, european_countries, refugee_countries, other_countries)

# Add a new row for Germany
new_row = {
    'Staatsangehörigkeit': 'Deutschland',
    '2023': 71855657,
    '2022': 72034650,
    '2021': 72344071,
    '2020': 72569978,
    '2019': 72768689
}

df = add_new_row(df, new_row)


In [11]:

# Function to melt the population DataFrame
def melt_population_data(df_population):
    return df_population.melt(id_vars=['Staatsangehörigkeit'], var_name='Year', value_name='Population')

# Function to parse and extract relevant components from column names
def parse_column_names(df, prefix=""):
    crime_cols = df.columns[1:]  # Assuming the first column is 'Crime'
    parsed_columns = crime_cols.str.extract(r'(?P<Country>.*)_(?P<Year>\d{4})')
    parsed_columns['Original'] = crime_cols
    parsed_columns['Country'] = prefix + parsed_columns['Country']  # Adding prefix if needed
    return parsed_columns

# Function to merge crime data with population data
def merge_data(df_population, data_combined_countries, parsed_columns):
    # Ensure 'Year' is string for accurate merging
    df_population['Year'] = df_population['Year'].astype(str)
    
    # Initialize a result DataFrame
    result_data = {'Crime': data_combined_countries['Crime'].values}
    
    # Iterate over each row in the parsed columns DataFrame
    for _, row in parsed_columns.iterrows():
        country = row['Country']
        year = row['Year']
        original_col = row['Original']
        
        # Find the matching population
        matched_population = df_population[
            (df_population['Staatsangehörigkeit'] == country) & (df_population['Year'] == year)
        ]
        
        if not matched_population.empty:
            population = matched_population['Population'].values[0]
            # Calculate the crime rate per population
            result_data[original_col] = data_combined_countries[original_col] / population

    return pd.DataFrame(result_data)

# Assuming df_population and data_combined_countries are predefined
df_population_melted = melt_population_data(df)
parsed_columns = parse_column_names(data_combined_countries)

# Merge the datasets
df_result = merge_data(df_population_melted, data_combined_countries, parsed_columns)


In [12]:

def melt_dataframe(df):
    # Use pd.melt to pivot the DataFrame
    melted_df = pd.melt(df.reset_index(), id_vars='Crime', value_vars=df.columns,
                        var_name='Year_Variable', value_name='Value')

    # Extract the year and the country from the 'Year_Variable' column
    melted_df['Year'] = melted_df['Year_Variable'].str.extract(r'(\d{4})$')
    melted_df['Country'] = melted_df['Year_Variable'].str.replace(r'_\d{4}$', '')

    # Remove the 'Year_Variable' column
    return melted_df.drop(columns=['Year_Variable'])

def rearrange_and_filter(df):
    # Rearrange the columns for better readability
    df = df[['Crime', 'Year', 'Country', 'Value']]

    # Create a pivot table with 'Year' and 'Crime' as the index and 'Country' as the columns
    df_pivoted = df.pivot_table(index=['Year', 'Crime'], columns='Country', values='Value').reset_index()

    # Filter out specific crime categories
    excluded_crimes = {'Abandonment', 'Arson', 'Smuggling of Persons', 'Resisting Arrest'}
    return df_pivoted[~df_pivoted['Crime'].isin(excluded_crimes)]

# Assuming df_result is already defined
df_to_pivot = df_result.copy()
df_melted = melt_dataframe(df_to_pivot)

# Save the tidy DataFrame to a CSV file
df_melted.to_csv('tidy_crime_data.csv', index=False)

df_pivoted_back_ = rearrange_and_filter(df_melted)
df_pivoted_back_.to_csv('pivot_crime_data.csv', index=False)


/var/folders/qg/sgj3q0zj6xvcgvl5h1k6w_nm0000gn/T/ipykernel_79233/4080438250.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  melted_df['Country'] = melted_df['Year_Variable'].str.replace(r'_\d{4}$', '')


In [13]:

def merge_and_sum_crimes(df, crime1, crime2, suffixes=('_Homicide', '_Murder')):
    # Filter DataFrames for specified crimes
    df_crime1 = df[df['Crime'] == crime1]
    df_crime2 = df[df['Crime'] == crime2]
    
    # Merge the two dataframes on 'Year' and 'Crime'
    df_merged = pd.merge(df_crime1, df_crime2, on=['Year'], suffixes=suffixes)

    # Define the countries and regions for which to sum values
    countries = ['Deutschland', 'Afghanistan', 'Bulgarien', 'China', 'Frankreich',
                 'Griechenland', 'Indien', 'Irak', 'Italien', 'Kosovo', 'Kroatien',
                 'Niederlande', 'Polen', 'Portugal', 'Rumänien', 'Russische Föderation',
                 'Serbien', 'Spanien', 'Syrien', 'Türkei', 'Ukraine', 'Ungarn',
                 'Österreich', 'Europäische Länder', 'Asylsuchende', 'Andere Länder']
    
    # Sum the values for Homicide and Murder across specified columns
    for country in countries:
        df_merged[country] = df_merged[f'{country}{suffixes[0]}'] + df_merged[f'{country}{suffixes[1]}']
        df_merged = df_merged.drop(columns=[f'{country}{suffixes[0]}', f'{country}{suffixes[1]}'])

    # Select relevant columns and set the Crime column
    df_sum = df_merged[['Year'] + countries]
    df_sum['Crime'] = f'{crime1}-{crime2}'

    return df_sum

# Assuming df_pivoted_back_ is predefined
# Merge and sum data for 'Homicide' and 'Murder'
df_sum = merge_and_sum_crimes(df_pivoted_back_, 'Homicide', 'Murder')

# Filter out the original 'Homicide' and 'Murder' rows and append the new summed data
df_pivoted_back_ = df_pivoted_back_[(df_pivoted_back_['Crime'] != 'Homicide') & (df_pivoted_back_['Crime'] != 'Murder')]
df_pivoted_back_ = pd.concat([df_pivoted_back_, df_sum], ignore_index=True)

# Optionally, save the final DataFrame to CSV
df_pivoted_back_.to_csv('final_crime_data.csv', index=False)


/var/folders/qg/sgj3q0zj6xvcgvl5h1k6w_nm0000gn/T/ipykernel_79233/1282352720.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sum['Crime'] = f'{crime1}-{crime2}'


In [14]:
# Ensure the original lists aren't modified by creating new lists that include the additional fields
def update_country_list(original_list, extra_fields):
    return original_list + extra_fields

# Define additional fields
additional_fields = ['Year', 'Crime', 'Deutschland']

# Update lists with additional fields for CSV output
european_countries_extended = update_country_list(european_countries, additional_fields + ['Europäische Länder'])
refugee_countries_extended = update_country_list(refugee_countries, additional_fields + ['Asylsuchende'])
other_countries_extended = update_country_list(other_countries, additional_fields + ['Andere Länder'])
df_extended = pd.concat([df_pivoted_back_, df_sum], ignore_index=True)

# Assuming df_extended is already defined and correctly concatenated
df_extended.to_csv('states_germany_crimes.csv', index=False)

# Export specific subsets of the data to separate CSV files
df_extended[european_countries_extended].to_csv('EU_states_germany_crimes.csv', index=False)
df_extended[refugee_countries_extended].to_csv('asyl_states_germany_crimes.csv', index=False)
df_extended[other_countries_extended].to_csv('other_states_german_crimes.csv', index=False)
